# Importation des librairies

In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

import importlib
spec = importlib.util.spec_from_file_location("preprocessing", "..\\utils\\preprocessing.py")
preprocessing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocessing)

spec = importlib.util.spec_from_file_location("fspliter", "..\\utils\\files_spliter.py")
fspliter = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fspliter)

spec = importlib.util.spec_from_file_location("results", "..\\utils\\results.py")
results = importlib.util.module_from_spec(spec)
spec.loader.exec_module(results)

# Lecture des données et preprocessing

In [ ]:
data = fspliter.get_mice(0)
data = fspliter.retrieve_day(data, 3)
data = preprocessing.do_preprocessing(data, 'RN')
data = preprocessing.transform_state_to_binary(data , 'RN')
data = data.drop(['filename'], axis=1)
data

In [ ]:
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range=(0, 1))
data[data.columns] = scaler.fit_transform(data[data.columns])
data

# Séparation en sets d'entrainement et de test

In [ ]:
# train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data['state'])
#serparte the features and the labels in train and test frist 20% of the data are for test
train = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

#state column as label and the others as features
train_features = train.drop(['state'], axis=1)
train_labels = train['state']

test_features = test.drop(['state'], axis=1)
test_labels = test['state']

# Model

In [ ]:
BATCH_SIZE = 20
model = MLPClassifier(
  hidden_layer_sizes = (16),
  activation = 'relu',
  solver = 'adam',
  alpha = 1e-5,
  batch_size = BATCH_SIZE,
  learning_rate = 'constant',
  verbose = True
)
model.fit(train_features, train_labels)
predict = model.predict(test_features)

# Résultats

In [ ]:
results.scores(test_labels, predict, ['n', 'r'])